In [15]:
!git clone https://github.com/Tanveer2719/NIDS_Coursework.git

Cloning into 'NIDS_Coursework'...
remote: Enumerating objects: 115, done.
remote: Counting objects: 100% (115/115), done.
remote: Compressing objects: 100% (78/78), done.
remote: Total 115 (delta 38), reused 111 (delta 34), pack-reused 0 (from 0)
Receiving objects: 100% (115/115), 139.96 KiB | 2.00 MiB/s, done.
Resolving deltas: 100% (38/38), done.


In [16]:
%cd /kaggle/working/NIDS_Coursework/My_Code
!ls

/kaggle/working/NIDS_Coursework/My_Code
classification.py    main.ipynb       preprocess.py  record_level_embedding.py
featureSelection.py  NIDS_Coursework  __pycache__    transformer.py


In [17]:
%rm -r __pycache__

In [18]:
!git pull origin main

From https://github.com/Tanveer2719/NIDS_Coursework
 * branch            main       -> FETCH_HEAD
Already up to date.


In [19]:
import pandas as pd
import numpy as np

path = "/kaggle/input/unsw-nb-15/UNSW-NB15_combined.csv"

df = pd.read_csv(path)

print(df.shape)
df = df.drop(columns=['id', 'label'])
print(df.shape)

print(f"🧾 Unique class labels: {df['attack_cat'].unique()}")
df['label'] = df['attack_cat'].apply(lambda x: 0 if x == 'Normal' else 1)

print(f'Unique values for label {df["label"].unique()}')

df = df.drop(columns=["attack_cat"])
print(f'new shape{df.shape}')

print(df.dtypes)


(257673, 45)
(257673, 43)
🧾 Unique class labels: ['Normal' 'Backdoor' 'Analysis' 'Fuzzers' 'Shellcode' 'Reconnaissance'
 'Exploits' 'DoS' 'Worms' 'Generic']
Unique values for label [0 1]
new shape(257673, 43)
dur                  float64
proto                 object
service               object
state                 object
spkts                  int64
dpkts                  int64
sbytes                 int64
dbytes                 int64
rate                 float64
sttl                   int64
dttl                   int64
sload                float64
dload                float64
sloss                  int64
dloss                  int64
sinpkt               float64
dinpkt               float64
sjit                 float64
djit                 float64
swin                   int64
stcpb                  int64
dtcpb                  int64
dwin                   int64
tcprtt               float64
synack               float64
ackdat               float64
smean                  int64
dmean   

In [20]:
# perform preprocessing

from preprocess import Preprocess

# separate the features and labels so that the labesl are not encoded
labels = df['label']
features = df.drop(columns=['label'])

pp = Preprocess()
processed_features = pp.fit_transform_df_auto(df = features,n_categorical_levels=32, expected_categorical_format='onehot')

print(processed_features.shape)
processed_features.head()

Encoding the 32 levels for proto
Encoding the 13 levels for service
Encoding the 11 levels for state
(257673, 95)


,dur,spkts,dpkts,sbytes,dbytes,rate,sttl,dttl,sload,dload,...,state_2,state_3,state_4,state_5,state_6,state_7,state_8,state_9,state_10,state_11
0,0.027889,0.193225,0.172921,0.331292,0.312312,0.312595,0.997874,1.000000,0.424562,0.534539,...,False,False,False,False,False,False,False,False,False,False
1,0.121803,0.284598,0.393619,0.398471,0.645181,0.316704,0.747160,0.998579,0.401347,0.775718,...,False,False,False,False,False,False,False,False,False,False
2,0.234590,0.224248,0.304405,0.353884,0.574953,0.196832,0.747160,0.998579,0.326962,0.650937,...,False,False,False,False,False,False,False,False,False,False
3,0.239956,0.267974,0.275582,0.388675,0.402879,0.194440,0.747160,0.998579,0.351625,0.479722,...,False,False,False,False,False,False,False,False,False,False
4,0.090294,0.248312,0.209072,0.378428,0.339064,0.256038,0.999294,0.998579,0.402218,0.489854,...,False,False,False,False,False,False,False,False,False,False


In [21]:
# concat the features and the labels
processed_full_df = pd.concat([processed_features, labels], axis=1)
processed_full_df.head()

,dur,spkts,dpkts,sbytes,dbytes,rate,sttl,dttl,sload,dload,...,state_3,state_4,state_5,state_6,state_7,state_8,state_9,state_10,state_11,label
0,0.027889,0.193225,0.172921,0.331292,0.312312,0.312595,0.997874,1.000000,0.424562,0.534539,...,False,False,False,False,False,False,False,False,False,0
1,0.121803,0.284598,0.393619,0.398471,0.645181,0.316704,0.747160,0.998579,0.401347,0.775718,...,False,False,False,False,False,False,False,False,False,0
2,0.234590,0.224248,0.304405,0.353884,0.574953,0.196832,0.747160,0.998579,0.326962,0.650937,...,False,False,False,False,False,False,False,False,False,0
3,0.239956,0.267974,0.275582,0.388675,0.402879,0.194440,0.747160,0.998579,0.351625,0.479722,...,False,False,False,False,False,False,False,False,False,0
4,0.090294,0.248312,0.209072,0.378428,0.339064,0.256038,0.999294,0.998579,0.402218,0.489854,...,False,False,False,False,False,False,False,False,False,0


In [24]:
from featureSelection import PSOFeatureSelector

pso = PSOFeatureSelector()
selected_mask, selected_feature = pso.run_pso(df=processed_full_df, target_column='label', verbose=True)

selected_feature


=== Iteration 1/10 ===
Particle 1/20: Score = 0.8844  --> New personal best! Previous: -inf
*** New global best score found: 0.8844 (Particle 1) ***
Particle 2/20: Score = 0.8875  --> New personal best! Previous: -inf
*** New global best score found: 0.8875 (Particle 2) ***
Particle 3/20: Score = 0.8805  --> New personal best! Previous: -inf
Particle 4/20: Score = 0.8887  --> New personal best! Previous: -inf
*** New global best score found: 0.8887 (Particle 4) ***
Particle 5/20: Score = 0.8886  --> New personal best! Previous: -inf
Particle 6/20: Score = 0.8775  --> New personal best! Previous: -inf
Particle 7/20: Score = 0.8768  --> New personal best! Previous: -inf
Particle 8/20: Score = 0.8846  --> New personal best! Previous: -inf
Particle 9/20: Score = 0.8861  --> New personal best! Previous: -inf
Particle 10/20: Score = 0.8857  --> New personal best! Previous: -inf
Particle 11/20: Score = 0.8850  --> New personal best! Previous: -inf
Particle 12/20: Score = 0.8947  --> New pers

Index(['sbytes', 'dbytes', 'rate', 'sttl', 'dttl', 'sload', 'dload', 'sloss',
       'dloss', 'dinpkt', 'sjit', 'swin', 'dtcpb', 'synack', 'smean', 'dmean',
       'response_body_len', 'ct_state_ttl', 'ct_src_dport_ltm',
       'ct_dst_src_ltm', 'is_ftp_login', 'ct_ftp_cmd', 'proto_1', 'proto_2',
       'proto_3', 'proto_4', 'proto_7', 'proto_10', 'proto_11', 'proto_12',
       'proto_13', 'proto_14', 'proto_17', 'proto_19', 'proto_22', 'proto_24',
       'proto_26', 'proto_27', 'proto_28', 'proto_29', 'proto_30', 'service_1',
       'service_7', 'service_11', 'state_1', 'state_3', 'state_5', 'state_6',
       'state_7', 'state_8', 'state_9', 'state_11'],
      dtype='object')

In [22]:
columns_to_keep = ['sbytes', 'dbytes', 'rate', 'sttl', 'dttl', 'sload', 'dload', 'sloss',
       'dloss', 'dinpkt', 'sjit', 'swin', 'dtcpb', 'synack', 'smean', 'dmean',
       'response_body_len', 'ct_state_ttl', 'ct_src_dport_ltm',
       'ct_dst_src_ltm', 'is_ftp_login', 'ct_ftp_cmd', 'proto_1', 'proto_2',
       'proto_3', 'proto_4', 'proto_7', 'proto_10', 'proto_11', 'proto_12',
       'proto_13', 'proto_14', 'proto_17', 'proto_19', 'proto_22', 'proto_24',
       'proto_26', 'proto_27', 'proto_28', 'proto_29', 'proto_30', 'service_1',
       'service_7', 'service_11', 'state_1', 'state_3', 'state_5', 'state_6',
       'state_7', 'state_8', 'state_9', 'state_11']

In [23]:
df_selected = processed_full_df[columns_to_keep].copy()
df_selected.shape

(257673, 52)

In [24]:
df_selected.dtypes

sbytes               float64
dbytes               float64
rate                 float64
sttl                 float64
dttl                 float64
sload                float64
dload                float64
sloss                float64
dloss                float64
dinpkt               float64
sjit                 float64
swin                 float64
dtcpb                float64
synack               float64
smean                float64
dmean                float64
response_body_len    float64
ct_state_ttl         float64
ct_src_dport_ltm     float64
ct_dst_src_ltm       float64
is_ftp_login         float64
ct_ftp_cmd           float64
proto_1                 bool
proto_2                 bool
proto_3                 bool
proto_4                 bool
proto_7                 bool
proto_10                bool
proto_11                bool
proto_12                bool
proto_13                bool
proto_14                bool
proto_17                bool
proto_19                bool
proto_22      

In [25]:
from record_level_embedding import RecordLevelEmbedder
numerical_columns = [
   'sbytes','dbytes','rate','sttl','dttl','sload','dload','sloss','dloss','dinpkt','sjit','swin','dtcpb','synack','smean','dmean','response_body_len',
    'ct_state_ttl', 'ct_src_dport_ltm','ct_dst_src_ltm','is_ftp_login','ct_ftp_cmd']

embedder = RecordLevelEmbedder(selected_df = df_selected, numerical_columns = numerical_columns, embed_dimension= 64)

embedded_df = embedder.transform_to_df()

embedded_df.head()

[INFO] Initializing RecordLevelEmbedder...
[INFO] Detecting categorical blocks...
[INFO] Detected 3 categorical blocks:
   - proto: 19 columns
   - service: 3 columns
   - state: 8 columns
[INFO] Embedder initialized with embedding dimension 64.
[INFO] Building Keras embedding model...
   - Adding numerical input: sbytes
   - Adding numerical input: dbytes
   - Adding numerical input: rate
   - Adding numerical input: sttl
   - Adding numerical input: dttl
   - Adding numerical input: sload
   - Adding numerical input: dload
   - Adding numerical input: sloss
   - Adding numerical input: dloss
   - Adding numerical input: dinpkt
   - Adding numerical input: sjit
   - Adding numerical input: swin
   - Adding numerical input: dtcpb
   - Adding numerical input: synack
   - Adding numerical input: smean
   - Adding numerical input: dmean
   - Adding numerical input: response_body_len
   - Adding numerical input: ct_state_ttl
   - Adding numerical input: ct_src_dport_ltm
   - Adding numeric

,embed_0,embed_1,embed_2,embed_3,embed_4,embed_5,embed_6,embed_7,embed_8,embed_9,...,embed_54,embed_55,embed_56,embed_57,embed_58,embed_59,embed_60,embed_61,embed_62,embed_63
0,-0.517890,0.389708,-0.415776,-0.193178,-0.362159,-1.079117,-0.427850,0.026337,0.457210,-0.185582,...,-0.952518,-0.375622,0.786778,-0.470167,-0.101331,0.896281,-0.312254,-0.316843,0.504701,0.143516
1,-0.453383,0.531504,-0.554560,-0.151107,-0.499106,-1.101279,-0.460603,0.119242,0.436695,-0.125054,...,-0.987737,-0.320164,0.839663,-0.477795,-0.246207,0.857436,-0.321194,-0.438306,0.614095,0.057004
2,-0.480481,0.504831,-0.611274,-0.271098,-0.473030,-1.139839,-0.440460,0.070507,0.603166,-0.147639,...,-0.867667,-0.207396,0.809151,-0.415374,-0.182325,1.026290,-0.350676,-0.419966,0.647681,0.095222
3,-0.620863,0.227751,-0.532981,-0.274855,-0.182096,-1.074285,-0.512655,-0.034567,0.326234,-0.111355,...,-0.802008,-0.384400,0.786532,-0.275384,0.070809,0.650666,-0.218445,-0.501259,0.539020,-0.070217
4,-0.463144,0.468762,-0.545000,-0.453566,-0.373296,-1.260049,-0.640231,-0.148146,0.617678,-0.034824,...,-0.749090,-0.124910,0.788831,-0.410715,-0.103532,1.169846,-0.310665,-0.436652,0.452920,0.204747


In [26]:
# Binary classification
import numpy as np
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.models import Model


print(processed_full_df['label'].unique())

binary_target = (processed_full_df['label'] != 0).astype(int).values
unique, counts = np.unique(binary_target, return_counts=True)
print("Classes and their counts:")
for cls, cnt in zip(unique, counts):
    print(f"Class {cls}: {cnt}")

[0 1]
Classes and their counts:
Class 0: 93000
Class 1: 164673


In [27]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.models import Model
from transformer import BasicTransformer
from classification import LastTokenClassificationHead
from sklearn.model_selection import train_test_split



# input data
X_embed = embedded_df.values                   # shape: (num_samples, embed_dim)
X_embed_seq = tf.expand_dims(X_embed, axis=1)  # shape: (batch_size, 1, embed_dim)

input_layer = Input(shape=(1, X_embed.shape[1]), name="record_input")

# Create transformer
transformer = BasicTransformer(
    n_layers=2,
    internal_size=128,
    n_heads=4,
    verbose=True  # See progress!
)


x = transformer.apply(input_layer, training=True)


# Create head
classification_head = LastTokenClassificationHead()

# Pool last token
x = classification_head.apply(x)

#  Add MLP head  
for i, layer_size in enumerate([128]):
    x = Dense(layer_size, activation="relu", name=f"classification_mlp_{i}_{layer_size}")(x)
    x = Dropout(0.1)(x)

#  output layer for binary classification
output_layer = Dense(1, activation="sigmoid", name="binary_output")(x)

# Build model  
model = Model(inputs=input_layer, outputs=output_layer)
model.summary()

# Compile model 
model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"],
    jit_compile=True
)

# split data

X_np = X_embed_seq.numpy() if hasattr(X_embed_seq, 'numpy') else np.array(X_embed_seq)


X_train, X_val, y_train, y_val = train_test_split(
    X_np, binary_target, test_size=0.2, random_state=42, stratify=binary_target
)

# Train
model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=20,
    batch_size=32
)

[BasicTransformer] Applying Encoder Block 1/2
[EncoderBlock] Input shape: (None, 1, 64)
[EncoderBlock] Output shape: (None, 1, 64)
[EncoderBlock] Input shape: (None, 1, 64)
[EncoderBlock] Output shape: (None, 1, 64)
[BasicTransformer] Applying Encoder Block 2/2
[EncoderBlock] Input shape: (None, 1, 64)
[EncoderBlock] Output shape: (None, 1, 64)
[EncoderBlock] Input shape: (None, 1, 64)
[EncoderBlock] Output shape: (None, 1, 64)
[BasicTransformer] Final output shape: (None, 1, 64)


Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ record_input (InputLayer)       │ (None, 1, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block_0_transformer_encoder     │ (None, 1, 64)          │       149,504 │
│ (TransformerEncoderBlock)       │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block_1_transformer_encoder     │ (None, 1, 64)          │       149,504 │
│ (TransformerEncoderBlock)       │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ slice_last (Lambda)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ classification_mlp_0_128        │ (None, 128)            │         8,320 │
│ (Dense)                         │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ binary_output (Dense)           │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 307,457 (1.17 MB)

 Trainable params: 307,457 (1.17 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
[EncoderBlock] Input shape: (None, 1, 64)
[EncoderBlock] Output shape: (None, 1, 64)
[EncoderBlock] Input shape: (None, 1, 64)
[EncoderBlock] Output shape: (None, 1, 64)
[EncoderBlock] Input shape: (None, 1, 64)
[EncoderBlock] Output shape: (None, 1, 64)
[EncoderBlock] Input shape: (None, 1, 64)
[EncoderBlock] Output shape: (None, 1, 64)
6442/6442 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8965 - loss: 0.2058[EncoderBlock] Input shape: (None, 1, 64)
[EncoderBlock] Output shape: (None, 1, 64)
[EncoderBlock] Input shape: (None, 1, 64)
[EncoderBlock] Output shape: (None, 1, 64)
6442/6442 ━━━━━━━━━━━━━━━━━━━━ 44s 5ms/step - accuracy: 0.8965 - loss: 0.2058 - val_accuracy: 0.9183 - val_loss: 0.1694
Epoch 2/20
6442/6442 ━━━━━━━━━━━━━━━━━━━━ 27s 4ms/step - accuracy: 0.9115 - loss: 0.1756 - val_accuracy: 0.9157 - val_loss: 0.1727
Epoch 3/20
6442/6442 ━━━━━━━━━━━━━━━━━━━━ 27s 4ms/step - accuracy: 0.9150 - loss: 0.1696 - val_accuracy: 0.9219 - val_loss: 0.1628
Epoch 4/20
6442/6442 

In [28]:
# evaluate the model
results = model.evaluate(X_val, y_val)
print(f"Validation loss: {results[0]:.4f}, Validation accuracy: {results[1]:.4f}")


1611/1611 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9321 - loss: 0.1361
Validation loss: 0.1364, Validation accuracy: 0.9323


In [31]:
from sklearn.metrics import f1_score, confusion_matrix
import numpy as np

# Predict probabilities
y_pred_prob = model.predict(X_val)
y_pred = (y_pred_prob >= 0.5).astype(int).flatten()

# F1
f1 = f1_score(y_val, y_pred)
print(f"F1 Score: {f1:.4f}")

# Confusion matrix
cm = confusion_matrix(y_val, y_pred, labels=[0, 1])
print("Confusion Matrix:\n", cm)

if cm.shape == (2, 2):
    tn, fp, fn, tp = cm.ravel()
    detection_rate = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    false_alarm_rate = fp / (fp + tn) if (fp + tn) > 0 else 0.0
    print(f"Detection Rate (Recall): {detection_rate:.4f}")
    print(f"False Alarm Rate: {false_alarm_rate:.4f}")
else:
    print("Confusion matrix does not contain both classes!")
    print("Check your labels: y_val:", np.unique(y_val), " y_pred:", np.unique(y_pred))


1611/1611 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step
F1 Score: 0.9469
Confusion Matrix:
 [[16905  1695]
 [ 1795 31140]]
Detection Rate (Recall): 0.9455
False Alarm Rate: 0.0911
